In [ ]:
import pandas as pd

# Desafio MTEch

**Descrição**: Sua equipe faz parte de uma empresa de consultoria, a MTEch Consulting Solutions, que foi contratada por uma agência de Turismo do Rio de Janeiro, a Viajar+, para ajudar na formulação de novos pacotes de viagem e impedir sua falência. Essa agência sofreu muito com a pandemia e precisa urgentemente de ajuda para retomar as atividades, vendendo, em 2021, pacotes de viagens para 2023.

**O desafio então é que você e sua equipe criem 1 pacote de viagem diferenciado e inovador**, justamente para que a Viajar+ possa atrair novos clientes e se destacar no
mercado mais uma vez.

## Perfil Alvo
Uma família, contendo um casal de 30 anos e 1 filha de 5 anos.

**Ponto turístico obrigatório pré-definido:** Museu de Belas Artes.

### Pontos chave

- Primeira vigem
- Criança adora praia
- Casal apaixonado por arte
- Conhecer história do Rio

### Descrição de acomodação ideal

- Bairro tranquilo e seguro
- Acesso à internet
- Apartamento inteiro
- Perto da praia

### Orçamento

- Máximo de 1500 reais
- Diária + Taxas
- Entrada em pontos turísticos
- Transporte de taxí ou aplicativo (ida e volta | 3 reais  p/ Km)

  
## Referências: 
[Preços Museu Belas Artes](https://mnba.gov.br/portal/museu/localizacao#:~:text=Ter%C3%A7a%20a%20sexta%2Dfeira%20das,a%20R%24%208%2C00.)

[Bairros barulhentos no Rio](https://extra.globo.com/noticias/rio/leitores-apontam-os-locais-mais-barulhentos-do-rio-especialista-faz-alerta-para-estresse-291108.html)

[Guia do NPS](https://resultadosdigitais.com.br/blog/o-que-e-nps/)


In [ ]:
# Montando drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Explorando dataset
df = pd.read_csv('/content/drive/MyDrive/MTEch/desafio/base_desafio_MTEch_2021.csv')
display(df.head(3))
display(df.info())

# Reestruturando campos
df['superhost'].replace(('t', 'f'), (1, 0), inplace=True)
df['host_tem_foto_perfil'].replace(('t', 'f'), (1, 0), inplace=True)
df['host_tem_identidade_verificada'].replace(('t', 'f'), (1, 0), inplace=True)
df['localização_é_exata'].replace(('t', 'f'), (1, 0), inplace=True)

df['preço'] = df['preço'].str.split("$", n = 1, expand = True).values[:,1]
df['taxa_de_limpeza'] = df['taxa_de_limpeza'].str.split("$", n = 1, expand = True).values[:,1]
df['preço'] = pd.to_numeric(df['preço'], errors='coerce') # podemos ignorar os erros pois valores acima de mil reais 
df['taxa_de_limpeza'] = pd.to_numeric(df['taxa_de_limpeza'], errors='coerce') # não são relevantes para nosso perfil alvo

,id,link_da_acomodacão,nome,resumo,o_espaço,descrição,visao_geral_bairro,observações,transito,acesso,interacao,regras_da_casa,link_da_foto,host_id,host_url,nome_host,inicio_host,superhost,numero_de_acomodacoes_host,verificações_do_host,host_tem_foto_perfil,host_tem_identidade_verificada,bairro,cidade,estado,CEP,localização_é_exata,tipo_de_propriedade,tipo_de_quarto,banheiros,quartos,camas,facilidades,metros_quadrados,preço,taxa_de_limpeza,pessoas_inclusas_no_preço,noites_mínimas,noites_máximas,número de reviews,primeira_review,última_review,avaliação_geral,avaliação_limpeza,avaliação_checkin,avaliacao_comunicação_host,avaliação_localização,avaliações_por_mês
0,17878,https://www.airbnb.com/rooms/17878,Very Nice 2Br - Copacabana - WiFi,Pls note that special rates apply for Carnival...,- large balcony which looks out on pedestrian ...,Pls note that special rates apply for Carnival...,This is the best spot in Rio. Everything happe...,NaN,Excellent location. Close to all major public ...,The entire apartment is yours. It's like your ...,I will be available throughout your stay shoul...,Please leave the apartment in a clean fashion ...,https://a0.muscache.com/im/pictures/65320518/3...,68997,https://www.airbnb.com/users/show/68997,Matthias,2010-01-08,t,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,Copacabana,Rio de Janeiro,Rio de Janeiro,22020-050,t,Condominium,Entire home/apt,1.0,2.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$332.00,$378.00,2,4,30,243,2010-07-15,2019-11-11,93.0,10.0,10.0,10.0,10.0,2.13
1,25026,https://www.airbnb.com/rooms/25026,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",This newly renovated studio (last renovations ...,"Our apartment is a little gem, everyone loves ...",Copacabana is a lively neighborhood and the ap...,"For any stay superior to 15 days, an additiona...",At night we recommend you to take taxis only. ...,"internet wi-fi, cable tv, air cond, ceiling fa...","Only at check in, we like to leave our guests ...",Smoking outside only. Family building so pleas...,https://a0.muscache.com/im/pictures/3003965/68...,102840,https://www.airbnb.com/users/show/102840,Viviane,2010-04-03,f,3.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,Copacabana,Rio de Janeiro,Rio de Janeiro,22060-020,t,Apartment,Entire home/apt,1.0,1.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$160.00,$250.00,2,7,60,235,2010-06-07,2019-10-08,94.0,10.0,9.0,10.0,10.0,2.04
2,31560,https://www.airbnb.com/rooms/31560,NICE & COZY 1BDR - IPANEMA BEACH,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,Die Nachbarschaft von Ipanema ist super lebend...,NaN,"Bus, U-Bahn, Taxi und Leihfahrräder in der Nähe.","Die Urlauber dürfen das Badezimmer benutzen, d...",NaN,"So far, I haven't had any problems with guests...",https://a0.muscache.com/im/pictures/83114449/2...,135635,https://www.airbnb.com/users/show/135635,Renata,2010-05-31,t,1.0,"['email', 'phone', 'manual_online', 'facebook'...",t,t,Ipanema,Rio de Janeiro,RJ,22410-003,t,Apartment,Entire home/apt,1.0,1.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$273.00,$84.00,2,2,1125,271,2010-07-11,2019-11-21,96.0,10.0,10.0,10.0,10.0,2.38


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33715 entries, 0 to 33714
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              33715 non-null  int64  
 1   link_da_acomodacão              33715 non-null  object 
 2   nome                            33654 non-null  object 
 3   resumo                          31661 non-null  object 
 4   o_espaço                        20756 non-null  object 
 5   descrição                       32785 non-null  object 
 6   visao_geral_bairro              18838 non-null  object 
 7   observações                     11247 non-null  object 
 8   transito                        18739 non-null  object 
 9   acesso                          16032 non-null  object 
 10  interacao                       16802 non-null  object 
 11  regras_da_casa                  17424 non-null  object 
 12  link_da_foto                    

None

## 1. Passo (avaliações)

A base de dados apresenta mais de 30 mil registros, decidimos começar filtrando pelas avaliações da acomodação. Para isso utilizamos a metodologia NPS (Net Promoter Score).

Essa metodologia parte do seguinte princípio: Em uma escala de 0 a 10, quanto você recomendaria a Empresa X para um amigo? 

- Nota de 0 à 6: **detratores** = não ficaram satisfeitos
- Nota de 7 à 8: **neutros** = não ficaram insatisfeitos mas nao são leais
- Nota de 9 à 10: **promotores** = recomendariam a empresa

Então excluímos todas acomodações com menos de 9 pontos nas avaliações disponíveis e reduzimos a amostra em aproximadamente 57%.

In [ ]:
# Separando as melhores avaliações baseadas no NPS (Net Promoter Score)
df_melhores_avaliacoes = df.loc[(df['avaliação_localização'] >= 9) & 
                                (df['avaliação_limpeza'] >= 9) & 
                                (df['avaliação_checkin'] >= 9) & 
                                (df['avaliacao_comunicação_host'] >= 9) &
                                (df['avaliação_geral'] >= 90)]
df_melhores_avaliacoes.shape

(14305, 48)

## 2. Passo (necessidades)

Para satisfazer nosso perfil-alvo devemos levar em consideração seus requisitos.
- 4 Dias de hospedagem
- 3 pessoas inclusas no preço
- 2 camas (1 de solteiro e 1 de casal)
- Internet (Wifi)

Com esses filtros conseguimos diminuir o número de acomodações possíveis para **192**.

In [ ]:
# Filtrando acomodações com duas camas (1 solteiro e 2 casal),  3 pessoas inclusas e 4 noites mínimas para a estadia
df_acomodacoes = df_melhores_avaliacoes.loc[(df_melhores_avaliacoes['camas'] == 2) &
                                            (df_melhores_avaliacoes['noites_mínimas'] <= 4) &
                                            (df_melhores_avaliacoes['pessoas_inclusas_no_preço'] == 3)]

# Filtrando apenas acomodaçoes com wifi
df_acomodacoes = df_acomodacoes.loc[df_acomodacoes['facilidades'].astype(str).str.contains('Wifi')]

df_acomodacoes.shape

(192, 48)

## 3. Passo (segurança e conforto)

Tendo em vista que nosso perfil-alvo é uma família, achamos interessante remover acomodações de quartos privados para trazer mais conforto.

Excluímos também alguns bairros conhecidos por serem mais agitados e barulhentos.

Pensando na segurança e dedicação do host, escolhemos acomodações com hosts de identidade verificada.

Percebemos que a base de dados contia acomodações fora da cidade do Rio então também às excluímos e o resultado final foi de **17** acomodações.

In [ ]:
# Filtrando apartamentos inteiros
df_acomodacoes = df_acomodacoes.loc[df_acomodacoes['tipo_de_quarto'] != 'Private room']

# Pensando na segurança
df_acomodacoes = df_acomodacoes.loc[df_acomodacoes['host_tem_identidade_verificada'] == 1]

# Removendo bairros barulhentos
df_acomodacoes = df_acomodacoes.loc[(df_acomodacoes['bairro'] != 'Copacabana') &
                                    (df_acomodacoes['bairro'] != 'Ipanema') &
                                    (df_acomodacoes['bairro'] != 'Leblon') &
                                    (df_acomodacoes['bairro'] != 'Barra da Tijuca')]

# Removendo acomodações fora da cidade do Rio
df_acomodacoes = df_acomodacoes.loc[df_acomodacoes['cidade'] == 'Rio de Janeiro']

df_acomodacoes.shape

(17, 48)

## 4. Passo (preços)

Estipulamos um valor de 1000 reais para gastar com estadia (4 diárias + taxa de limpeza). E então excluímos acomodações que poderiam não caber no orçamento.

Neste ponto restaram apenas **5** acomodações para serem avaliadas.

In [ ]:
# Definindo 1000 reais para gastar com estadia
df_acomodacoes = df_acomodacoes.loc[(df_acomodacoes['preço']*4 + df_acomodacoes['taxa_de_limpeza'] <= 1000)]

df_acomodacoes.shape

(5, 48)

## 5. Passo (escolha da acomodação)

Para escolha da acomodação ideal ordenamos os resultados pelo número de reviews e pela avaliação geral.

Visitamos todos os links e o lugar que mais agradou foi, coincidentemente, o primeiro do ranking.

Além de ser o melhor avaliado, a localização é boa, possuí dois quartos e tem uma vista ótima.

In [ ]:
# Ordenando acomodações por reviews
df_acomodacoes = df_acomodacoes.sort_values(['número de reviews', 'avaliação_geral'], ascending=False)

# Analisando acomodações restantes
display(df_acomodacoes)
list(df_acomodacoes['link_da_acomodacão'])

,id,link_da_acomodacão,nome,resumo,o_espaço,descrição,visao_geral_bairro,observações,transito,acesso,interacao,regras_da_casa,link_da_foto,host_id,host_url,nome_host,inicio_host,superhost,numero_de_acomodacoes_host,verificações_do_host,host_tem_foto_perfil,host_tem_identidade_verificada,bairro,cidade,estado,CEP,localização_é_exata,tipo_de_propriedade,tipo_de_quarto,banheiros,quartos,camas,facilidades,metros_quadrados,preço,taxa_de_limpeza,pessoas_inclusas_no_preço,noites_mínimas,noites_máximas,número de reviews,primeira_review,última_review,avaliação_geral,avaliação_limpeza,avaliação_checkin,avaliacao_comunicação_host,avaliação_localização,avaliações_por_mês
5992,5837343,https://www.airbnb.com/rooms/5837343,Great 2br apt w/ view of Corcovado,"Sunny non-smoking apt on a quiet street, with ...","This apartment is located on a nice, quiet res...","Sunny non-smoking apt on a quiet street, with ...",Botafogo is a charming tree-lined neighbourhoo...,"Dies ist kein Haus zu mieten, es ist die zweit...",Within a walking distance to a subway station ...,You will have the entire apartment for yoursel...,I'm happy to help you with any issues you have...,We kindly ask our guests not to smoke in the a...,https://a0.muscache.com/im/pictures/80751452/0...,29342871,https://www.airbnb.com/users/show/29342871,Magnus,2015-03-14,0.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1.0,1.0,Botafogo,Rio de Janeiro,Rio de Janeiro,22260-020,1,Apartment,Entire home/apt,3.0,2.0,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,151.0,150.0,3,2,40,88,2015-06-22,2019-10-07,98.0,10.0,10.0,10.0,10.0,1.63
8689,10491000,https://www.airbnb.com/rooms/10491000,Rio - Estúdio com vista para o Aterro do Flamengo,Realmente não há como não se emocionar com a b...,"Vista privilegiada do Aterro do Flamengo, onde...",Realmente não há como não se emocionar com a b...,O viajante/hóspede poderá apreciar aspectos ge...,NaN,Farta disponibilidade de transporte público co...,Todas as dependências do imóvel estão disponív...,Moro muito próxima ao apartamento e em caso de...,1-É permitido 1 cão (até 10 kg) desde que os m...,https://a0.muscache.com/im/pictures/ed679454-d...,53966937,https://www.airbnb.com/users/show/53966937,Ana,2016-01-09,1.0,7.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",1.0,1.0,Flamengo,Rio de Janeiro,Rio de Janeiro,22210-030,1,Apartment,Entire home/apt,1.0,0.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,97.0,120.0,3,2,89,44,2016-04-25,2019-11-14,100.0,10.0,10.0,10.0,10.0,1.01
7342,9056293,https://www.airbnb.com/rooms/9056293,New Studio Rio de Janeiro South side,"Beautiful and modern Studio, fully equipped w...","The Studio is very well located, close to tra...","Beautiful and modern Studio, fully equipped w...",Laranjeiras e' um bairro tranquilo com farto c...,access to the Studio is done thru a side hallw...,Metrô a 10 minutos a pé. Pontos de onibus par...,The guests will have access through the side e...,We love to host and meet people from around th...,Solicitamos a gentileza de não fumar dentro do...,https://a0.muscache.com/im/pictures/d411462c-9...,47246718,https://www.airbnb.com/users/show/47246718,Maria Das Gracas,2015-10-23,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",1.0,1.0,Laranjeiras,Rio de Janeiro,Rio de Janeiro,22221-140,0,Guesthouse,Entire home/apt,1.0,1.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,181.0,160.0,3,2,30,36,2015-12-12,2019-10-31,100.0,10.0,10.0,10.0,9.0,0.75
11110,12265532,https://www.airbnb.com/rooms/12265532,Best Bedroom/Living Room - ALL Remodeled,"One-bedroom apartment, all remodeled. Brand ne...","All new!! Refrigerator, stove with grill, 42"" ...","One-bedroom apartment, all remodeled. Brand ne...",NaN,NaN,Located 5 min. walking to two Metro stations w...,The building has no external recreational area...,24 hr availability for any need/questions.,Não aceitamos animais domésticos. Aceitamos no...,https://a0.muscache.com/im/pictures/a09acdd7-8...,

['https://www.airbnb.com/rooms/5837343',
 'https://www.airbnb.com/rooms/10491000',
 'https://www.airbnb.com/rooms/9056293',
 'https://www.airbnb.com/rooms/12265532',
 'https://www.airbnb.com/rooms/16193187']

In [ ]:
# Nossa escolha
print('Nossa escolha de acomodação ideal foi:')
acomodacao_ideal = df_acomodacoes.iloc[0]
print(acomodacao_ideal)
acomodacao_ideal.facilidades

Nossa escolha de acomodação ideal foi:
id                                                                          5837343
link_da_acomodacão                             https://www.airbnb.com/rooms/5837343
nome                                             Great 2br apt w/ view of Corcovado
resumo                            Sunny non-smoking apt on a quiet street, with ...
o_espaço                          This apartment is located on a nice, quiet res...
descrição                         Sunny non-smoking apt on a quiet street, with ...
visao_geral_bairro                Botafogo is a charming tree-lined neighbourhoo...
observações                       Dies ist kein Haus zu mieten, es ist die zweit...
transito                          Within a walking distance to a subway station ...
acesso                            You will have the entire apartment for yoursel...
interacao                         I'm happy to help you with any issues you have...
regras_da_casa                    We 

'{TV,Internet,Wifi,"Air conditioning",Kitchen,"Free parking on premises",Doorman,Elevator,"Free street parking","Buzzer/wireless intercom",Washer,"Smoke detector","Carbon monoxide detector","First aid kit",Essentials,"24-hour check-in",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","translation missing: en.hosting_amenity_49","translation missing: en.hosting_amenity_50","Hot water","Bed linens",Microwave,"Coffee maker",Refrigerator,"Dishes and silverware","Cooking basics",Oven,Stove,"Patio or balcony"}'